In [26]:
# imports

import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import warnings
pd.options.mode.chained_assignment = None
warnings.simplefilter(action='ignore', category=FutureWarning)

# Reading in the Data

Before we can do any data analysis, we need to be able to actually find and read in data. After doing some research, we found a website that basically had an archive for data for multiple sports (including baseball). Because we couldn't just download the datasets and because we didn't want to manually type out the datapoints, we decided to utilize web scraping to read in the data for us.

The first step was to come up with all the links that we were planning to scrape. Because the first page of the search result contained hyperlinks to all the other pages of the search result, we scraped the first page for all the HTMLs and then created a list of URLs to scrape.

In [29]:
# base link used for each page 
base_link = 'https://www.prosportstransactions.com/baseball/Search/' 
# link to the first page of the search result 
link = 'https://www.prosportstransactions.com/baseball/Search/SearchResults.php?Player=&Team=&BeginDate=2000-01-01&EndDate=2019-11-19&DLChkBx=yes&submit=Search'



In [30]:
r = requests.get(link)
urlText = r.text

In [31]:
soup = BeautifulSoup(urlText, 'html.parser') # reads everything into something more readable than HTML 

In [32]:
all_links  = soup.find_all('a') # finds all links within the page 

In [33]:
# removes first and last 4 links, which are extraneous links
important_links = [x.get('href') for x in all_links[4:-4]] 

# adds base URL to each link to make each link a proper URL link for each page 
final_links = [link]
final_links = final_links+[base_link+x for x in important_links]

final_links contains the final list of all the URLs we're planning to scrape. We can then create a method that will loop through the links and properly scrape the data and concatenate it onto a dataframe. This method is timed and only used once: delays are set so that the website isn’t overloaded with traffic, and once the method runs through once the dataset gets saved as a csv to ensure that the website isn’t abused.

In [34]:
def number_cycles(links, n=3,sleep_time=2):
    '''
    This method takes in a list of links and scrapes the data, neatly organizing it into a dataframe. 
    Parameters: 
        Links: the list of URL links that you want to scrape 
        n: the number of links you want to read in from the list, default 3 
        sleep_time: the time the loop waits before reading in the website, default 2 seconds, TIME SHOULD BE INCREASED
            IF LENGTH OF LIST OF URLS IS LARGE
    Returns: 
        returns the dataframe containing the data scraped from all the links from the inputted list 
    '''
    final_df = pd.DataFrame()
    for i in range(n):
        print('working on page '+str(i))
        sleep(sleep_time)
        test = pd.read_html(links[i],header = 0)[0]
        final_df = pd.concat([final_df, test],ignore_index = True)
    return final_df

In [35]:
df = number_cycles(final_links,20,sleep_time=0)

working on page 0
working on page 1
working on page 2
working on page 3
working on page 4
working on page 5
working on page 6
working on page 7
working on page 8
working on page 9
working on page 10
working on page 11
working on page 12
working on page 13
working on page 14
working on page 15
working on page 16
working on page 17
working on page 18
working on page 19


In [36]:
df.to_csv('baseball_datasets/webscraped_dataset.csv')

In [37]:
df.head()

,Date,Team,Acquired,Relinquished,Notes
0,2000-03-23,Rockies,NaN,• John Thomson,placed on 60-day DL recovering from surgery on...
1,2000-04-01,Devil Rays,NaN,• Wilson Alvarez,placed on DL
2,2000-04-01,Dodgers,NaN,• Antonio Osuna,placed on 15-day DL
3,2000-04-08,Mets,NaN,• Darryl Hamilton,placed on 15-day DL with sprained left foot
4,2000-04-08,Orioles,NaN,• Mike Timlin,placed on 15-day DL with strained abdominal


Now we have a dataset that we can use! We can begin to analyze and clean it. 

# Understanding and Cleaning the Dataset 

In [38]:
# reading in CSV 
df = pd.read_csv('baseball_datasets/webscraped_dataset.csv')

In [39]:
def dot_remover(x):
    '''
    '''
    if not (x == x):
        return x 
    else: 
        return x[2:]
    
def dot_cleaner(df):
    df['Acquired'] = df['Acquired'].apply(dot_remover)
    df['Relinquished'] = df['Relinquished'].apply(dot_remover)
    return df

In [40]:
no_dots_df = dot_cleaner(df)

In [41]:
no_dots_df['Date'] = pd.to_datetime(no_dots_df['Date'])

In [42]:
no_dots_df.head()

,Unnamed: 0,Date,Team,Acquired,Relinquished,Notes
0,0,2000-03-23,Rockies,NaN,John Thomson,placed on 60-day DL recovering from surgery on...
1,1,2000-04-01,Devil Rays,NaN,Wilson Alvarez,placed on DL
2,2,2000-04-01,Dodgers,NaN,Antonio Osuna,placed on 15-day DL
3,3,2000-04-08,Mets,NaN,Darryl Hamilton,placed on 15-day DL with sprained left foot
4,4,2000-04-08,Orioles,NaN,Mike Timlin,placed on 15-day DL with strained abdominal


In [43]:
no_dots_df

,Unnamed: 0,Date,Team,Acquired,Relinquished,Notes
0,0,2000-03-23,Rockies,NaN,John Thomson,placed on 60-day DL recovering from surgery on...
1,1,2000-04-01,Devil Rays,NaN,Wilson Alvarez,placed on DL
2,2,2000-04-01,Dodgers,NaN,Antonio Osuna,placed on 15-day DL
3,3,2000-04-08,Mets,NaN,Darryl Hamilton,placed on 15-day DL with sprained left foot
4,4,2000-04-08,Orioles,NaN,Mike Timlin,placed on 15-day DL with strained abdominal
5,5,2000-04-08,Padres,NaN,Carlos Hernandez,placed on 15-day DL with strained groin
6,6,2000-04-08,Cubs,Glenallen Hill Sr.,NaN,activated from 15-day DL
7,7,2000-05-24,Braves,NaN,Kevin McGlinchy,placed on 15-day DL
8,8,2000-05-24,Indians,NaN,Paul Shuey,placed on 15-day DL
9,9,2000-05-24,Giants,Ellis Burks,NaN,activated from 15-day DL


In [44]:
# things to do: 
# save df as a external csv so that you don't have to constantly load it 
# rotate dataframe so that people injured and returned are all in one row

In [45]:
no_dots_df_copy = no_dots_df.copy()

In [46]:
no_dots_df_copy.head()

,Unnamed: 0,Date,Team,Acquired,Relinquished,Notes
0,0,2000-03-23,Rockies,NaN,John Thomson,placed on 60-day DL recovering from surgery on...
1,1,2000-04-01,Devil Rays,NaN,Wilson Alvarez,placed on DL
2,2,2000-04-01,Dodgers,NaN,Antonio Osuna,placed on 15-day DL
3,3,2000-04-08,Mets,NaN,Darryl Hamilton,placed on 15-day DL with sprained left foot
4,4,2000-04-08,Orioles,NaN,Mike Timlin,placed on 15-day DL with strained abdominal


In [47]:
# you need to drop columns with NaN in acquired

In [53]:
import numpy as np
abcd =  no_dots_df_copy.replace(np.nan, '', regex=True)
no_dots_df_copy['person'] = abcd['Acquired']+abcd['Relinquished']

In [49]:
len(no_dots_df_copy['person'].unique())

300

In [52]:
final_df = pd.DataFrame()
names = no_dots_df_copy['person'].unique()
list_o_shit = []
for i in range(len(names)):
    name_df = no_dots_df_copy[((no_dots_df_copy['Acquired']==names[i]) | 
                            (no_dots_df_copy['Relinquished']==names[i]))]
    def is_relinquished(x):
        if x == x: 
            return 0 
        else: 
            return 1
    name_df['order stuff'] = name_df['Acquired'].apply(is_relinquished)
    index_stuff = list(name_df['order stuff'])
    list_fuckups = []
    for i in range(len(index_stuff)-1):
        if index_stuff[i] == index_stuff[i+1]:
            list_fuckups.append((i+(i+1))/2)
    for i in list_fuckups: 
        line = pd.DataFrame({"Date": np.nan, "Team": np.nan,'Acquired':np.nan, 'Relinquished':np.nan,
                        'Notes': np.nan, 'person':np.nan, 'number_rows': np.nan,'order stuff':(i-.5)//2}, index=[i])
        name_df = name_df.append(line, ignore_index=False)
        name_df = name_df.sort_index().reset_index(drop=True)
    if name_df.iloc[0]['Acquired'] == name_df.iloc[0]['Acquired']: # if acquired is first data point (no data on previous injury )
        name_df = name_df[1:]
    left = name_df[name_df['order stuff'] == 1.0].reset_index()
    right = name_df[name_df['order stuff'] == 0][['Date','Notes']].reset_index()
    new_df = pd.concat([left,right],axis = 1)
    list_o_shit.append(new_df)
    final_df = pd.concat([final_df,new_df])
final_df.columns = ['useless1','Date Relinquished','Team','useless2','Player','Injury','useless3',
                   'useless4','useless5','useless6','Date Returned','Notes about Return']
final_df = final_df[['Date Relinquished','Team','Player','Injury','Date Returned','Notes about Return']]

AssertionError: Number of manager items must equal union of block items
# manager items: 10, # tot_items: 13

In [51]:
final_df.head()

,index,Unnamed: 0,Date,Team,Acquired,Relinquished,Notes,person,order stuff,index,Date,Notes
0,0,0,2000-03-23,Rockies,NaN,John Thomson,placed on 60-day DL recovering from surgery on...,John Thomson,1,234.0,2001-05-12,activated from DL
1,290,290,2001-05-25,Rockies,NaN,John Thomson,placed on 15-day DL,John Thomson,1,NaN,NaT,NaN


In [19]:
final_df['duration of injury'] = final_df['Date Returned']-final_df['Date Relinquished']
final_df.head()

KeyError: 'Date Returned'

In [196]:
final_df['duration of injury'].iloc[1].days

3

In [197]:
def temp(x):
    return x.days
final_df['duration of injury integer'] = final_df['duration of injury'].apply(temp)

In [198]:
final_df.head()

,Date Relinquished,Team,Player,Injury,Date Returned,Notes about Return,duration of injury,duration of injury integer
0,2018-08-09,Dodgers,Yasiel Puig,oblique injury (DTD),NaT,NaN,NaT,NaN
1,2019-05-21,Reds,Yasiel Puig,sprained right shoulder (DTD),2019-05-24,returned to lineup,3 days,3.0
0,2018-09-30,Indians,Yan Gomes,right thumb injury (DTD),2018-10-05,returned to lineup,5 days,5.0
1,2019-05-07,Nationals,Yan Gomes,bruised forearm (DTD),2019-05-08,returned to lineup,1 days,1.0
0,2018-08-07,Reds,Joey Votto,knee injury (DTD),2018-08-10,returned to lineup,3 days,3.0


In [199]:
final_df[final_df['duration of injury integer']==302]

,Date Relinquished,Team,Player,Injury,Date Returned,Notes about Return,duration of injury,duration of injury integer
1,2018-09-07,Cardinals,Yadier Molina,left hamstring injury (DTD),2019-07-06,returned to lineup,302 days,302.0


In [253]:
final_df[final_df['Player'] == 'Yadier Molina']

,Date Relinquished,Team,Player,Injury,Date Returned,Notes about Return,duration of injury
0,2018-09-07,Cardinals,Yadier Molina,left hamstring injury (DTD),NaT,NaN,NaT
1,2019-05-29,Cardinals,Yadier Molina,bruised right hand (DTD),2018-09-13,returned to lineup,-258 days
2,2019-07-04,Cardinals,Yadier Molina,right thumb injury (DTD),2019-07-06,returned to lineup,2 days
3,2019-07-07,Cardinals,Yadier Molina,right thumb injury (DTD),NaT,NaN,NaT


In [202]:
no_dots_df_copy[no_dots_df_copy['person']=='Yadier Molina']

,Date,Team,Acquired,Relinquished,Notes,person,number_rows
352,2018-09-07,Cardinals,NaN,Yadier Molina,left hamstring injury (DTD),Yadier Molina,6
353,2018-09-13,Cardinals,Yadier Molina,NaN,returned to lineup,Yadier Molina,6
354,2019-05-29,Cardinals,NaN,Yadier Molina,bruised right hand (DTD),Yadier Molina,6
355,2019-07-04,Cardinals,NaN,Yadier Molina,right thumb injury (DTD),Yadier Molina,6
356,2019-07-06,Cardinals,Yadier Molina,NaN,returned to lineup,Yadier Molina,6
357,2019-07-07,Cardinals,NaN,Yadier Molina,right thumb injury (DTD),Yadier Molina,6


In [190]:
final_df['duration of injury'].value_counts()

1 days      168
2 days      121
3 days       72
4 days       32
5 days       25
8 days        9
6 days        8
7 days        6
9 days        4
0 days        4
13 days       3
10 days       3
11 days       2
14 days       2
302 days      1
241 days      1
213 days      1
17 days       1
344 days      1
199 days      1
41 days       1
18 days       1
91 days       1
12 days       1
271 days      1
239 days      1
Name: duration of injury, dtype: int64

In [258]:
test = no_dots_df_copy[((no_dots_df_copy['Acquired']=='Yadier Molina') | (no_dots_df_copy['Relinquished']=='Yadier Molina'))]
test.head()


,Date,Team,Acquired,Relinquished,Notes,person,number_rows
352,2018-09-07,Cardinals,NaN,Yadier Molina,left hamstring injury (DTD),Yadier Molina,6
353,2018-09-13,Cardinals,Yadier Molina,NaN,returned to lineup,Yadier Molina,6
354,2019-05-29,Cardinals,NaN,Yadier Molina,bruised right hand (DTD),Yadier Molina,6
355,2019-07-04,Cardinals,NaN,Yadier Molina,right thumb injury (DTD),Yadier Molina,6
356,2019-07-06,Cardinals,Yadier Molina,NaN,returned to lineup,Yadier Molina,6


In [259]:
def is_relinquished(x):
    if x == x: 
        return 0 
    else: 
        return 1
test['order stuff'] = test['Acquired'].apply(is_relinquished)
index_stuff = list(test['order stuff'])
list_fuckups = []
for i in range(len(index_stuff)-1):
    if index_stuff[i] == index_stuff[i+1]:
        list_fuckups.append((i+(i+1))/2)
list_fuckups

[2.5]

In [260]:

for i in list_fuckups: 
    line = pd.DataFrame({"Date": np.nan, "Team": np.nan,'Acquired':np.nan, 'Relinquished':np.nan,
                        'Notes': np.nan, 'person':np.nan, 'number_rows': np.nan,'order stuff':(i-.5)%2}, index=[i])
    test = test.reset_index().append(line, ignore_index=False)
    test = test.sort_index().reset_index(drop=True)
test


,Acquired,Date,Notes,Relinquished,Team,index,number_rows,order stuff,person
0,NaN,2018-09-07 00:00:00,left hamstring injury (DTD),Yadier Molina,Cardinals,352.0,6.0,1.0,Yadier Molina
1,Yadier Molina,2018-09-13 00:00:00,returned to lineup,NaN,Cardinals,353.0,6.0,0.0,Yadier Molina
2,NaN,2019-05-29 00:00:00,bruised right hand (DTD),Yadier Molina,Cardinals,354.0,6.0,1.0,Yadier Molina
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,NaN,2019-07-04 00:00:00,right thumb injury (DTD),Yadier Molina,Cardinals,355.0,6.0,1.0,Yadier Molina
5,Yadier Molina,2019-07-06 00:00:00,returned to lineup,NaN,Cardinals,356.0,6.0,0.0,Yadier Molina
6,NaN,2019-07-07 00:00:00,right thumb injury (DTD),Yadier Molina,Cardinals,357.0,6.0,1.0,Yadier Molina


In [169]:
left = test[test['order stuff'] == 1.0].reset_index()

In [175]:
right = test[test['order stuff'] == 0][['Date','Notes']].reset_index()

In [176]:
right

,index,Date,Notes
0,0,2018-08-07,returned to lineup
1,2,NaT,NaN
2,4,2019-05-24,returned to lineup


In [177]:
pd.concat([left,right],axis = 1)

,index,Date,Team,Acquired,Relinquished,Notes,person,number_rows,order stuff,index,Date,Notes
0,1.0,2018-08-09,Dodgers,NaN,Yasiel Puig,oblique injury (DTD),Yasiel Puig,4.0,1.0,0,2018-08-07,returned to lineup
1,3.0,2019-05-21,Reds,NaN,Yasiel Puig,sprained right shoulder (DTD),Yasiel Puig,4.0,1.0,2,NaT,NaN
2,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,2019-05-24,returned to lineup


In [24]:
filler = pd.DataFrame(no_dots_df_copy['person'].value_counts()).reset_index()
filler

,index,person
0,Niko Goodrum,12
1,Khris Davis,11
2,Kris Bryant,10
3,Andrew Benintendi,10
4,Kevin Kiermaier,10
5,Yoan Moncada,10
6,Brandon Crawford,10
7,Dee Gordon,10
8,Tommy Pham,10
9,Avisail Garcia,10


In [25]:
no_dots_df_copy = no_dots_df_copy.merge(filler, left_on='person',right_on='index', copy = False)[
    ['Date','Team','Acquired','Relinquished','Notes','person_x','person_y']]
no_dots_df_copy.columns = ['Date','Team','Acquired','Relinquished','Notes','person','number_rows']
no_dots_df_copy.head()

,Date,Team,Acquired,Relinquished,Notes,person,number_rows
0,2018-08-07,Dodgers,Yasiel Puig,NaN,returned to lineup,Yasiel Puig,4
1,2018-08-09,Dodgers,NaN,Yasiel Puig,oblique injury (DTD),Yasiel Puig,4
2,2019-05-21,Reds,NaN,Yasiel Puig,sprained right shoulder (DTD),Yasiel Puig,4
3,2019-05-24,Reds,Yasiel Puig,NaN,returned to lineup,Yasiel Puig,4
4,2018-08-07,Indians,Yan Gomes,NaN,returned to lineup,Yan Gomes,5


In [ ]:
cleanest_rows = no_dots_df_copy[no_dots_df_copy['number_rows']==2]
cleanest_rows = cleanest_rows.dropna(subset=['Relinquished']).merge(cleanest_rows,left_on='Relinquished',right_on='Acquired')
cleanest_rows = cleanest_rows[['Date_x','Team_x','Relinquished_x','Notes_x','Date_y','Notes_y']]
cleanest_rows.columns = ['date injured','team','player','notes on injury','date returned','notes on return']
cleanest_rows = cleanest_rows.drop(cleanest_rows[cleanest_rows['date injured']>cleanest_rows['date returned']].index)
cleanest_rows.head()



In [ ]:
cleanest_rows['number of days injured'] = cleanest_rows['date returned']-cleanest_rows['date injured']
cleanest_rows.head()

In [ ]:
#extraneous_row = cleanest_rows[cleanest_rows['date injured']>cleanest_rows['date returned']]
#extraneous_row['date return']

In [ ]:
cleanest_rows['notes on injury'].unique()

In [ ]:
cleanest_rows.groupby('notes on injury')['number of days injured'].describe()

In [25]:
with open('untitled.txt') as f:
    content = f.readlines()

In [26]:
content = [x.strip() for x in content] 
label = [x.split()[0][:-1] for x in content]
description = [x.split(' ',1)[1:][0] for x in content]

In [27]:
pd.set_option('max_colwidth',1000)
pd.DataFrame({'label':label,'Description':description})

,label,Description
0,playerID,Player ID code
1,yearID,Year - the season in which the player was active
2,Stint,player's stint (order of appearances within a season)
3,teamID,Team - team player was in
4,lgID,League
5,G,Games - number of games played
6,AB,At Bats - number of at-bats
7,R,Runs - number of runs or scores player contributed to
8,H,Hits - contact with the ball results in player reaching first base
9,2B,Doubles - contact with ball results in player reaching second base
